In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import block_diag
from scipy.linalg import svdvals

General update according to 

$$U \Sigma V^\top + ab^\top$$

This can be changed to

$$m = U^\top a \qquad p = a-Um$$ 

$$n = V^\top b \qquad q = b-Vn$$

some notes: $m$ and $n$ represent vectors in the range/corange wheras $p$ and $q$ reprsent things that are not in the respecting co-range

This gives the following new SVD-problem:

$$
K=
\begin{bmatrix}
\sigma & 0\\
0& 0
\end{bmatrix}
+
\begin{bmatrix}
m\\
\|p\|
\end{bmatrix}
\begin{bmatrix}
n^\top& \|p\|
\end{bmatrix}
$$

# Some random ideas to get an cheap estimate of $\|K\|_*$

In [ ]:
N = 20
A = np.random.rand(N,N)

We now add a column.
This means setting
$$b^\top = [0,\dots,0,1]$$
and
$$a = c$$ 
where c is column

In [ ]:
U,s,Vt = np.linalg.svd(A[:,:-1],full_matrices=False) 

In [ ]:
b = np.zeros(N-1)
b[-1]=1
a=A[:,-1]

In [ ]:
K = np.zeros((s.size+1,s.size+1))
K[:-1,:-1]=np.diag(s)
m = U.T@a
K[:-1,-1]=m
K[-1,-1]=np.linalg.norm(a-U@m)

In [ ]:
plt.matshow(K)

In [ ]:
np.sum(s)

In [ ]:
np.linalg.norm(A,'nuc')

In [ ]:
np.trace(K)

In [ ]:
np.linalg.norm(K,'nuc')

In [ ]:
np.linalg.norm(K,'fro')

In [ ]:
np.trace(K)+np.linalg.norm(K[:-1,-1],1)

In [ ]:
np.trace(K)+np.sum(K[:-1,-1])

In [ ]:
X2 = 0.5*(0.5*(K+K.T)+K@K.T*np.diag(1/np.diag(K)))
plt.matshow(X2)
np.trace(X2)

In [ ]:
X = X2

In [ ]:
X = 0.5*(X+K@K.T*np.linalg.inv(X))
X@X-K@K.T

In [ ]:
np.linalg.inv?

# Remove collumn

In [ ]:
U,s,Vt = np.linalg.svd(A[:,:],full_matrices=False) 

In [ ]:
b = np.zeros(N)
b[-1]=1
a=-A[:,-1]
m = U.T@a
p = a-U@m
n = Vt@b
q = b-Vt.T@n

u = np.hstack([m,np.linalg.norm(p)])
v = np.hstack([n,np.linalg.norm(q)])

In [ ]:
K = np.zeros((s.size+1,s.size+1))
K[:-1,:-1]=np.diag(s)
K += u.reshape(-1,1)@v.reshape(1,-1)
K[-1,-1]=np.linalg.norm(a-U@m)

In [ ]:
plt.matshow(K)

In [ ]:
np.linalg.norm(A[:,:-1],'nuc')

In [ ]:
np.linalg.norm(K,'nuc')

In [ ]:
np.trace(K)

In [ ]:
np.linalg.norm(A,'nuc')

In [ ]:
#X2 = 0.5*(0.5*(K+K.T)+K@K.T*np.diag(1/np.diag(K)))
X2 = 0.5*(K+K@K.T*np.diag(1/np.diag(K)))
plt.matshow(X2)
np.trace(X2)

In [ ]:
def Bab_iteration(s,u,v):
    d_k = np.hstack([s,u[-1]*v[-1]]) #diagnal of K
    s_ = np.hstack([s,0])# elongated sigma vector
    return 0.5*d_k+ (0.5*d_k**2 + s_*u*v + 0.5*u**2*(v@v))/d_k



def est_add(U,s,Vt,c):
    """
    Function to estimate the add the column c
    """

    m = U.T@c
    p = c-U@m
    u = np.hstack([m,np.linalg.norm(p)])
    v = np.zeros_like(u)
    v[-1]=1
    
    return np.sum(Bab_iteration(s,u,v))
    

def est_remove(U,s,Vt,c,i_c):
    """
    Function to estimate the add the column c
    """
    b = np.zeros(Vt.shape[1])
    b[i_c]=1
    m = U.T@c
    p = c-U@m
    n = Vt@b #can probably be simplified
    q = b-Vt.T@n

    u = np.hstack([m,np.linalg.norm(p)])
    v = np.hstack([n,np.linalg.norm(q)])
    
    return np.sum(Bab_iteration(s,u,v))

In [ ]:
def norm_add(U,s,Vt,c):
    m = U.T@c
    p = c-U@m
    K = np.block([[np.diag(s),m.reshape(-1,1)],[np.zeros((1,len(s))),np.linalg.norm(p)]])
    return np.sum(svdvals(K,overwrite_a=True,check_finite=False))

def norm_remove(U,s,Vt,c,i_c):
    b = np.zeros(Vt.shape[1])
    b[i_c]=-1
    m = U.T@c
    p = c-U@m
    n = Vt@b#n = Vt[:,i_c]
    q = b-Vt.T@n

    u = np.hstack([m,np.linalg.norm(p)])
    v = np.hstack([n,np.linalg.norm(q)])
    
    K = u.reshape(-1,1)@v.reshape(1,-1)
    np.fill_diagonal(K, u*v+np.hstack([s,0]))
    #K = np.diag(np.hstack([s,0]))+u.reshape(-1,1)@v.reshape(1,-1)
    
    return np.linalg.norm(K,'nuc')#np.sum(svdvals(K,overwrite_a=True,check_finite=False))

Some Tests

In [ ]:
N = 20
A = np.random.rand(N,N)
U,s,Vt = np.linalg.svd(A,full_matrices=False)
print("||A||_* = ",np.linalg.norm(A,'nuc'))

In [ ]:
#add column
c = np.random.rand(N)

print("approx ||A||_* = ",est_add(U,s,Vt,c))
print("update ||A||_* = ",norm_add(U,s,Vt,c))
print("actual ||A||_* = ",np.linalg.norm(np.hstack([A,c.reshape(-1,1)]),'nuc'))

In [ ]:
#remove

i_c = -1
c = A[:,i_c]


print("approx ||A||_* = ",est_remove(U,s,Vt,c,i_c))
print("update ||A||_* = ",norm_remove(U,s,Vt,c,i_c))
print("actual ||A||_* = ",np.linalg.norm(A[:,:-1],'nuc'))

In [ ]:
b = np.zeros(Vt.shape[1])
b[i_c]=-1
A+c.reshape(-1,1)*b.reshape(1,-1)